# Test new IGV remote

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import socket
import os
import re

In [3]:
import pandas as pd

In [4]:
bam_paths_df = pd.read_csv('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/1k_genomes_bam_paths.txt', sep='\t', index_col=0)
bam_paths_df.index.name = 'patient_id'
bam_paths_df = bam_paths_df.reset_index()

In [5]:
socket.__file__

'/Users/cchu/opt/anaconda3/envs/mutation_reviewer_env/lib/python3.8/socket.py'

# test sockets

In [6]:
HOST = "127.0.0.1"  # The server's hostname or IP address
PORT = 60151  # The port used by the server
chromosome=17 
start_pos=7_569_720
end_pos = 7_592_868

In [7]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
# # s.recv(2000)
s.close()

In [8]:
s

<socket.socket [closed] fd=-1, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>

In [26]:
s.send('new \n '.encode('utf-8'))

6

In [27]:
s.recv(4096)

b'OK\n'

In [28]:
s.send('goto {}:{}-{} \n'.format(chromosome, start_pos, end_pos).encode('utf-8'))

25

In [29]:
s.recv(4096)

b'OK\n'

In [30]:
s.send('new\n '.encode('utf-8'))

5

In [31]:
s.recv(4096)

b'OK\n'

In [34]:
fn = bam_paths_df.iloc[0]['local_bam_path']
fn

'/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam'

In [40]:
s.send(f'load {fn} \n '.encode('utf-8'))

OSError: [Errno 9] Bad file descriptor

In [39]:
s.recv(4096)

OSError: [Errno 9] Bad file descriptor

In [11]:
from importlib.metadata import version

In [24]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
s.send('new \n '.encode('utf-8'))
print(s.recv(4096))
s.send('goto {}:{}-{} \n'.format(chromosome, start_pos, end_pos).encode('utf-8'))
print(s.recv(4096))
s.send(f'load {bam_paths_df.iloc[0]["local_bam_path"]} \n '.encode('utf-8'))
print(s.recv(4096))
s.close()

b'OK\n'
b'OK\n'
b'OK\n'


In [25]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
s.send('new \n '.encode('utf-8'))
print(s.recv(4096))
s.send('goto {}:{}-{} \n'.format(chromosome, start_pos, end_pos).encode('utf-8'))
print(s.recv(4096))
s.send(f'load {bam_paths_df.iloc[1]["local_bam_path"]} \n '.encode('utf-8'))
print(s.recv(4096))
s.close()

b'OK\n'
b'OK\n'
b'OK\n'


In [26]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))
s.send('new \n '.encode('utf-8'))
print(s.recv(4096))
s.send('goto {}:{}-{} \n'.format(chromosome, start_pos, end_pos).encode('utf-8'))
print(s.recv(4096))
s.send(f'load {bam_paths_df.iloc[0]["local_bam_path"]} \n '.encode('utf-8'))
print(s.recv(4096))
s.send(f'load {bam_paths_df.iloc[1]["local_bam_path"]} \n '.encode('utf-8'))
print(s.recv(4096))
s.close()

b'OK\n'
b'OK\n'
b'OK\n'
b'OK\n'


In [185]:
def _append_id(filename, id):
    return "{0}_{2}.{1}".format(*filename.rsplit('.', 1) + [id])

def _parse_loc(chromosome, pos1, pos2=None, expand=20):
    if expand < 20:
        print("IGV expands left and right margin by at least 20bp")
        expand=20
    start_pos = int(pos1-expand)
    if pos2 is None:
        end_pos = int(pos1+expand)
    else:
        end_pos = int(pos2)
    start_pos ='{:,}'.format(int(start_pos))
    end_pos = '{:,}'.format(int(end_pos))
    position= '{}:{}-{}'.format(chromosome, start_pos, end_pos)
    print("Position to view: {}".format(position))
    return position

VALID_VIEW_OPTS = ['squish', 'collapse', 'expand']

class igv_remote:

    def __init__(self, host="127.0.0.1", port=60151, view_type='collapse', sort='base', verbose=True, recv_timeout=60):
        self.set_viewopts(view_type,  sort)
        self.HOST = host
        self.PORT = port

        self.verboseprint = print if verbose else lambda *a, **k: None
        self.recv_timeout = recv_timeout # time to wait for a response from IGV before exiting (seconds)

    def set_viewopts(self, view_type,  sort):
        if view_type not in VALID_VIEW_OPTS:
            raise ValueError(f"view_type must be one of {VALID_VIEW_OPTS}. Input was {view_type}")
        self._view_type = view_type
        self._sort = sort

    def set_saveopts(self, img_dir, img_basename, img_init_id=0) :
        # check if path is absolute and exits
        if not os.path.exists(img_dir):
            print("Initializing a directory called {} in current dir".format(img_dir))
            os.mkdir(img_dir)
        img_fulldir = os.path.abspath(img_dir)
        print("Snapshots are available in {}".format(img_fulldir))
        # check if the image name has proper extension
        accepted_extensions = ["png", "svg", "jpg"]
        if not any(x in img_basename for x in accepted_extensions):
            raise ValueError("filename has to contain extension, one of jpg/svg/png")
        
        self._img_fulldir = img_fulldir
        self._img_basename = img_basename
        self._img_id = img_init_id
        
    def send_cmds(self, cmds):
        for cmd in cmds:
            self._send_cmd(cmd)

    def _send_cmd(self, cmd, timeout=20):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((self.HOST, self.PORT))
        
        format_cmd = f'{cmd} \n'.encode('utf-8')
        self.verboseprint(f"sending command: {format_cmd}")
        self.socket.send(format_cmd)

        self.socket.settimeout(timeout) # if IGV times out, we recommend you restart IGV
        self.verboseprint(self.socket.recv(4096))
        
        self.socket.close()

    def new(self):
        self._send_cmd('new')

    def load(self, *urls):
        print(urls)
        # self._send("new ")
        if len(urls) < 1:
            raise ValueError("Please provide at least one URL to load")
        for url in urls:
            self._send_cmd("load %s" % url)
            self._adjust_viewopts()

    def goto(self, chromosome, start_pos, end_pos=None, expand=20):
        """
        if only start_pos is supplied, we will expand view range by 'expand' parameter
        """

        position = _parse_loc(chromosome, start_pos, end_pos, expand)
        self._send_cmd( "goto %s" % position)
        # make sure viewopts are preserved
        self._adjust_viewopts()
    
    def goto_multiple(self, expand=20, **kwargs):
        """
        goto_multiple(expand=20, chr1=<seqname of first panel>, chr2=<seqname of second panel>, pos1=<position of first panel>, pos2=<position of second panel>)
        """
        chrpos = { "chr" : {}, "pos" : {} }
        try:
            for k, v in kwargs.items():
                print(k, v)
                arg = re.match(r'^(chr|pos)(\d+)$', k)
                if arg is not None:
                    chrpos[arg.group(1)][arg.group(2)] = v
                else:
                    raise Exception
            if chrpos["chr"].keys() != chrpos["pos"].keys():
                raise Exception
        except:
            raise ValueError("When specifying multiple loci, arguments must be of the format chr1 = <chr1>, pos1 = <pos1>, ..., chrN = <chrN>, posN = <posN>")

        print(chrpos)
        
        positions = []
        for (_, chrv), (_, posv) in zip(chrpos['chr'].items(), chrpos['pos'].items()):
            positions.append(_parse_loc(chrv, posv, None, expand))
        print(positions)
        self._send_cmd("goto {}".format(" ".join(positions)))
        # make sure viewopts are preserved
        self._adjust_viewopts()

    def snapshot(self): # snapshot as-is
        assert self._img_fulldir is not None, "Please set view optins with ir.set_saveopts() first"
        self._send_cmd( "snapshotDirectory %s" % self._img_fulldir)
        newname = _append_id(self._img_basename, self._img_id)
        self._send_cmd( "snapshot %s" % newname)
        self._img_id += 1

    def _adjust_viewopts(self):
        self._send_cmd(f"{self._view_type}")
        self._send_cmd(f"sort {self._sort}")



In [167]:
ir = igv_remote()

In [168]:
ir.set_saveopts(img_dir='test_igv_images', img_basename='test.png')

Snapshots are available in /Users/cchu/Desktop/Methods/MutationReviewer/example_notebooks/test_igv_images


stress test

In [169]:
import time
import tqdm

In [174]:
for i in tqdm.tqdm(range(10)):
    ir.new()
    # ir.goto(chromosome, start_pos, end_pos, expand=20)
    ir.goto_multiple(**dict(chr1=chromosome, chr2=chromosome, pos1=start_pos, pos2=end_pos))
    ir.load(bam_paths_df.iloc[0]["local_bam_path"])
    ir.load(bam_paths_df.iloc[1]["local_bam_path"])
    ir.snapshot()
    time.sleep(1)

  0%|                                                                                                                                       | 0/10 [00:00<?, ?it/s]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending c

 10%|████████████▋                                                                                                                  | 1/10 [00:01<00:10,  1.13s/it]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending c

 20%|█████████████████████████▍                                                                                                     | 2/10 [00:02<00:09,  1.13s/it]

...

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████▌                         | 8/10 [00:09<00:02,  1.13s/it]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending c

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 9/10 [00:10<00:01,  1.12s/it]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending c

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.13s/it]


# test with gsurls

In [177]:
import dalmatian

In [178]:
wm = dalmatian.WorkspaceManager('broad-getzlab-ibm-taml-t/Ebert-Sperling_MDS-AML-pran-5_Claudia-v2')
samples_df = wm.get_samples()

In [189]:
ir = igv_remote()
ir.set_saveopts(img_dir='test_igv_images', img_basename='test.png')

Snapshots are available in /Users/cchu/Desktop/Methods/MutationReviewer/example_notebooks/test_igv_images


In [191]:
for i in tqdm.tqdm(range(10)):
    ir.new()
    # ir.goto(chromosome, start_pos, end_pos, expand=20)
    ir.goto_multiple(**dict(chr1=chromosome, chr2=chromosome, pos1=start_pos, pos2=end_pos))
    ir.load(samples_df.iloc[0]["gpdw_DNA_WES_icev1_cram_or_bam_path"])
    ir.load(samples_df.iloc[1]["gpdw_DNA_WES_icev1_cram_or_bam_path"])
    ir.snapshot()
    time.sleep(1)

  0%|                                                                                                                                       | 0/10 [00:00<?, ?it/s]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam',)
sending command: b'load gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/046434_XQ_3239/v2/04

 10%|████████████▋                                                                                                                  | 1/10 [00:09<01:21,  9.10s/it]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam',)
sending command: b'load gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/046434_XQ_3239/v2/04

 20%|█████████████████████████▍                                                                                                     | 2/10 [00:15<01:01,  7.72s/it]

...

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████▌                         | 8/10 [00:55<00:13,  6.85s/it]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam',)
sending command: b'load gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/046434_XQ_3239/v2/04

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 9/10 [01:01<00:06,  6.51s/it]

sending command: b'new \n'
b'OK\n'
chr1 17
chr2 17
pos1 7569720
pos2 7592868
{'chr': {'1': 17, '2': 17}, 'pos': {'1': 7569720, '2': 7592868}}
Position to view: 17:7,569,700-7,569,740
Position to view: 17:7,592,848-7,592,888
['17:7,569,700-7,569,740', '17:7,592,848-7,592,888']
sending command: b'goto 17:7,569,700-7,569,740 17:7,592,848-7,592,888 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam',)
sending command: b'load gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/022613_PU_3426/v2/022613_PU_3426.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'
('gs://fc-02e4b730-cb29-48bb-9c3b-562e075457fe/Getz_Ebert_IBM_13-583_Exomes_10TNAsamples_November2020/RP-1886/Exome/046434_XQ_3239/v2/04

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:07<00:00,  6.73s/it]


# More tests

In [190]:
ir.new()

sending command: b'new \n'
b'OK\n'


In [143]:
ir.goto(chromosome, start_pos, end_pos, expand=20)

Position to view: 17:7,569,700-7,592,868
sending command: b'goto 17:7,569,700-7,592,868 \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'


In [144]:
ir.load(bam_paths_df.iloc[0]["local_bam_path"])

('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'


In [145]:
ir.load(bam_paths_df.iloc[1]["local_bam_path"])

('/Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam',)
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'


In [124]:
ir._adjust_viewopts()

sending command: b'collapse \n'
b'OK\n'
sending command: b'sort base \n'
b'OK\n'


In [88]:
ir.send_cmds(['new', 'goto {}:{}-{}'.format(chromosome, start_pos, end_pos), f'load {bam_paths_df.iloc[1]["local_bam_path"]}'])

sending command: b'new \n'
b'OK\n'
sending command: b'goto 17:7569720-7592868 \n'
b'OK\n'
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam \n'
b'OK\n'


In [89]:
ir.send_cmds(['new', 'goto {}:{}-{}'.format(chromosome, start_pos, end_pos), f'load {bam_paths_df.iloc[0]["local_bam_path"]}', f'load {bam_paths_df.iloc[1]["local_bam_path"]}'])

sending command: b'new \n'
b'OK\n'
sending command: b'goto 17:7569720-7592868 \n'
b'OK\n'
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00096.17_7571739_7590808.bam \n'
b'OK\n'
sending command: b'load /Users/cchu/Desktop/Methods/MutationReviewer/1k_genomes/HG00097.17_7571739_7590808.bam \n'
b'OK\n'
